In [ ]:
#一、训练词向量，将新测试集、旧测试集、训练集所有文本数据去重后训练词向量


In [ ]:
import numpy as np
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
import gensim
import os
import time
from gensim.models import doc2vec
from tqdm import tqdm
import logging
import gc
from gensim.models.word2vec import Word2Vec
import time

In [ ]:
#分块读取train数据


In [ ]:
train =pd.read_csv('/home/kesci/input/bytedance/train_final.csv',names=['qId','q','aId','a','target'],chunksize=100000000)


In [ ]:
i=1
#分块去重
time_start=time.time()
for data in train:
    print(i)
    sentence=pd.concat([data['q'],data['a']])
    sentence=sentence.drop_duplicates(["x"],keep="first")
    sentence.to_csv('/home/kesci/work/sentence/sentences'+str(i)+'.csv',index=False)
    del sentence

In [ ]:
sentence=pd.DataFrame()
for i in range(1,11):
    x=pd.read_csv('/home/kesci/work/sentence/sentences'+str(i)+'.csv')
    sentence=pd.concat(sentence,x)
    sentence=sentence.drop_duplicates(["x"],keep="first")
sentence.to_csv('/home/kesci/work/sentence/sentences_train.csv',index=False)

In [ ]:
test1=pd.read_csv('/home/kesci/input/bytedance/test_final_part1.csv',names=['qId','q','aId','a'])
test2 =pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv',names=['qId','q','aId','a'])
test1['a']=test1['a'].apply(lambda x:x[:-1])
test2['a']=test2['a'].apply(lambda x:x[:-1])
gc.collect()

In [ ]:
train=pd.read_csv('/home/kesci/work/sentence/sentences_train.csv')


In [ ]:
test=pd.DataFrame(pd.concat(test1['q'],test1['a'],test2['q'],test2['a'],train['x']))


In [ ]:
test.to_csv('/home/kesci/work/sentence/sentences_final.csv',index=False,header=0)


In [ ]:
sentences = gensim.models.word2vec.LineSentence('/home/kesci/work/sentence/sentences_final.csv')


In [ ]:
model= Word2Vec(size=200, window=5, min_count=3, workers=10,hs=1)


In [ ]:
model.build_vocab(sentences)


In [ ]:
class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
    '''用于保存模型, 打印损失函数等等'''
    def __init__(self, savedir, save_name="word2vector.model"):
        os.makedirs(savedir, exist_ok=True)
        self.save_path = os.path.join(savedir, save_name)
        self.epoch = 0
        self.pre_loss = 0
        self.best_loss = 999999999.9
        self.since = time.time()

    def on_epoch_end(self, model):
        self.epoch += 1
        cum_loss = model.get_latest_training_loss() # 返回的是从第一个epoch累计的
        epoch_loss = cum_loss - self.pre_loss
        time_taken = time.time() - self.since
        print("Epoch %d, loss: %.2f, time: %dmin %ds" % 
                    (self.epoch, epoch_loss, time_taken//60, time_taken%60))
        if self.best_loss > epoch_loss:
            self.best_loss = epoch_loss
            print("Better model. Best loss: %.2f" % self.best_loss)
            model.save(self.save_path)
            print("Model %s save done!" % self.save_path)

        self.pre_loss = cum_loss
        self.since = time.time()

In [ ]:
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)


In [ ]:
#训练词向量，当callbacks打印loss=0时，停止训练
savedir='/home/kesci/work/'
save_name='w2vfinal_all.model'
model.train(sentences, total_examples=model.corpus_count, 
                        epochs=40, compute_loss=True, report_delay=60*10, # 每隔10分钟输出一下日志
                        callbacks=[EpochSaver(savedir, save_name)])